In [41]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

In [42]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

import yfinance as yf
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import warnings
from dataclasses import asdict, is_dataclass, dataclass
import os
import json
import pickle
from pandas.api.types import is_datetime64_any_dtype
import numpy as np
from typing import Dict
import polars as pl

from src.common.AssetData import AssetData
from src.common.AssetDataPolars import AssetDataPolars
from src.common.AssetDataService import AssetDataService
from src.common.AssetFileInOut import AssetFileInOut 
from src.databaseService.OutsourceLoader import OutsourceLoader

from src.simulation.SimulatePortfolio import SimulatePortfolio
from src.strategy.StratBuyAndHold import StratBuyAndHold
from src.simulation.ResultAnalyzer import ResultAnalyzer
from src.common.AssetFileInOut import AssetFileInOut
from src.mathTools.CurveAnalysis import CurveAnalysis
from src.predictionModule.FourierML import FourierML
from src.common.YamlTickerInOut import YamlTickerInOut
from src.common.DataFrameTimeOperations import DataFrameTimeOperationsPandas as DFTO


In [43]:
assets=AssetFileInOut("../src/stockGroups/bin").loadDictFromFile("group_snp500_over20years")

# Convert to Polars for speedup
assetspl: Dict[str, AssetDataPolars] = {}
for ticker, asset in assets.items():
    assetspl[ticker] = AssetDataService.to_polars(asset)

In [44]:
asset = assetspl["MSFT"]

fin_quart = asset.financials_quarterly
fin_ann = asset.financials_annually
print(type(fin_quart["nonInterestIncome"].item(56)))
fin_ann.columns

<class 'str'>


['fiscalDateEnding',
 'reportedEPS',
 'reportedCurrency',
 'grossProfit',
 'totalRevenue',
 'costOfRevenue',
 'costofGoodsAndServicesSold',
 'operatingIncome',
 'sellingGeneralAndAdministrative',
 'researchAndDevelopment',
 'operatingExpenses',
 'investmentIncomeNet',
 'netInterestIncome',
 'interestIncome',
 'interestExpense',
 'nonInterestIncome',
 'otherNonOperatingIncome',
 'depreciation',
 'depreciationAndAmortization',
 'incomeBeforeTax',
 'incomeTaxExpense',
 'interestAndDebtExpense',
 'netIncomeFromContinuingOperations',
 'comprehensiveIncomeNetOfTax',
 'ebit',
 'ebitda',
 'netIncome']

In [36]:

# Convert all string columns to float, with errors resulting in null (NaN)
fin_ann = fin_ann.with_columns(
    [
        pl.col(col).cast(pl.Float64, strict=False)
        for col in fin_ann.columns
        if fin_ann[col].dtype == pl.Utf8
    ]
)

fin_quart = fin_quart.with_columns(
    [
        pl.col(col).cast(pl.Float64, strict=False)
        for col in fin_quart.columns
        if fin_quart[col].dtype == pl.Utf8
    ]
)

In [37]:
# Divide all numeric columns by "totalRevenue", handling missing values safely
fin_ann = fin_ann.with_columns(
    [
        (pl.col(col) / pl.col("totalRevenue")).alias(col)
        for col in fin_ann.columns
        if fin_ann[col].dtype.is_numeric() and col != "totalRevenue"
    ]
)

fin_quart = fin_quart.with_columns(
    [
        (pl.col(col) / pl.col("totalRevenue")).alias(col)
        for col in fin_quart.columns
        if fin_quart[col].dtype.is_numeric() and col != "totalRevenue"
    ]
)

In [38]:
for col, dtype in fin_ann.schema.items():
    print(f"{col}: {dtype}")

fiscalDateEnding: Datetime(time_unit='ns', time_zone='UTC')
reportedEPS: Float64
reportedCurrency: Float64
grossProfit: Float64
totalRevenue: Float64
costOfRevenue: Float64
costofGoodsAndServicesSold: Float64
operatingIncome: Float64
sellingGeneralAndAdministrative: Float64
researchAndDevelopment: Float64
operatingExpenses: Float64
investmentIncomeNet: Float64
netInterestIncome: Float64
interestIncome: Float64
interestExpense: Float64
nonInterestIncome: Float64
otherNonOperatingIncome: Float64
depreciation: Float64
depreciationAndAmortization: Float64
incomeBeforeTax: Float64
incomeTaxExpense: Float64
interestAndDebtExpense: Float64
netIncomeFromContinuingOperations: Float64
comprehensiveIncomeNetOfTax: Float64
ebit: Float64
ebitda: Float64
netIncome: Float64


In [39]:
a = fin_ann["fiscalDateEnding"].item(-1)
print(type(a))

fin_ann.with_columns(fin_ann["fiscalDateEnding"].map_elements(lambda x: pd.Timestamp(x), return_dtype=pd.Timestamp))

a = fin_ann["fiscalDateEnding"].item(-1)
print(type(a))

a = fin_quart["reportedDate"].item(-1)
print(type(a))

<class 'datetime.datetime'>
<class 'datetime.datetime'>
<class 'datetime.datetime'>


In [40]:
fin_quart = fin_quart.with_columns(
        (pl.col("surprisePercentage") / 1000.0).alias("surprisePercentage")
)